### Info
based on notebook3\
conversion of qkeras to hls4ml, predictions

## Create the hls_model

### Load packages and the model

In [1]:
import tensorflow as tf
import numpy as np
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits
co = {}; _add_supported_quantized_objects(co)

model_path = f"/eos/project/c/cms-l1ml/public/Pawel/axol1tl_model.h5"

model = tf.keras.models.load_model(model_path, custom_objects=co)
#model_config = model.get_config()

data_path = f"/eos/project/c/cms-l1ml/public/Pawel/complete.h5"
data_file = h5py.File(data_path, 'r')

2025-10-07 21:24:12.021007: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-07 21:24:12.055203: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-07 21:24:12.055264: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-07 21:24:12.055313: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-07 21:24:12.064319: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-07 21:24:12.064739: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

### Copy weights

In [2]:
ds_weights = data_file['model']['trimmed_encoder']['model_weights']
for layer in iter(model.layers):
    #print(layer.name)
    if layer.name in ('input_1','subtract','dot'): continue

    if layer.name == 'q_dense_4':
        weights_group = ds_weights['q_dense_4']['variational_auto_encoder']['vae__encoder']['q_dense_4']
        layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))
        continue

    weights_group = ds_weights[layer.name][layer.name]
    layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))

### Into hls4ml

In [3]:
import hls4ml

hls_config = hls4ml.utils.config_from_keras_model(
    model,
    granularity='name', 
    default_precision='fixed<18,8>',    # like the activation
    backend='Vitis')

/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/hls4ml/converters/__init__.py:27: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 57]], output shape: [None, 57]
Layer name: q_dense, layer type: QDense, input shapes: [[None, 57]], output shape: [None, 29]
Layer name: q_dense_1, layer type: QDense, input shapes: [[None, 29]], output shape: [None, 10]
Layer name: q_dense_2, layer type: QDense, input shapes: [[None, 10]], output shape: [None, 9]
Layer name: q_dense_3, layer type: QDense, input shapes: [[None, 9]], output shape: [None, 6]
Layer name: q_dense_4, layer type: QDense, input shapes: [[None, 6]], output shape: [None, 4]
Layer name: q_dense_6, layer type: QDense, input shapes: [[None, 4]], output shape: [None, 6]
Layer name: q_dense_7, layer type: QDense, input shapes: [[None, 6]], output shape: [None, 9]
Layer name: q_dense_8, layer type: QDense, input shapes: [[None, 9]], output shape: [None, 10]
Layer name: subtract, layer type: Merge, input shapes: [[None, 10], [None, 10]], output shape: [None, 10]
Layer name:

/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(


In [79]:
hls_config['LayerName']['dot']['Precision']['accum'] = 'fixed<18,14>'
hls_config

{'Model': {'Precision': 'fixed<18,8>',
  'ReuseFactor': 1,
  'Strategy': 'Latency',
  'BramFactor': 1000000000,
  'TraceOutput': False},
 'LayerName': {'input_1': {'Trace': False,
   'Precision': {'result': 'fixed<18,8>'}},
  'q_dense': {'Trace': False,
   'Precision': {'result': 'fixed<18,8>',
    'weight': 'fixed<6,3>',
    'bias': 'fixed<10,7>',
    'accum': 'fixed<18,8>'},
   'ReuseFactor': 1},
  'q_dense_quantized_relu': {'Trace': False,
   'Precision': {'result': 'ufixed<10,6,RND_CONV,SAT>',
    'table': 'fixed<18,8>'},
   'ReuseFactor': 1,
   'TableSize': 1024},
  'q_dense_1': {'Trace': False,
   'Precision': {'result': 'fixed<18,8>',
    'weight': 'fixed<6,3>',
    'bias': 'fixed<10,7>',
    'accum': 'fixed<18,8>'},
   'ReuseFactor': 1},
  'q_dense_1_quantized_relu': {'Trace': False,
   'Precision': {'result': 'ufixed<10,6,RND_CONV,SAT>',
    'table': 'fixed<18,8>'},
   'ReuseFactor': 1,
   'TableSize': 1024},
  'q_dense_2': {'Trace': False,
   'Precision': {'result': 'fixed<18

In [80]:
!rm -rf "new_hls_qkeras/"
!rm "new_hls_qkeras.tar.gz"

In [81]:
hls_model = hls4ml.converters.convert_from_keras_model(
    model, hls_config=hls_config, backend='Vitis',
    output_dir='./new_hls_qkeras'
)

Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 57]], output shape: [None, 57]
Layer name: q_dense, layer type: QDense, input shapes: [[None, 57]], output shape: [None, 29]
Layer name: q_dense_1, layer type: QDense, input shapes: [[None, 29]], output shape: [None, 10]
Layer name: q_dense_2, layer type: QDense, input shapes: [[None, 10]], output shape: [None, 9]
Layer name: q_dense_3, layer type: QDense, input shapes: [[None, 9]], output shape: [None, 6]
Layer name: q_dense_4, layer type: QDense, input shapes: [[None, 6]], output shape: [None, 4]
Layer name: q_dense_6, layer type: QDense, input shapes: [[None, 4]], output shape: [None, 6]
Layer name: q_dense_7, layer type: QDense, input shapes: [[None, 6]], output shape: [None, 9]
Layer name: q_dense_8, layer type: QDense, input shapes: [[None, 9]], output shape: [None, 10]
Layer name: subtract, layer type: Merge, input shapes: [[None, 10], [None, 10]], output shape: [None, 10]
Layer name:

/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(


In [82]:
hls_model.compile()

Writing HLS project


/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Done


## Test the models

In [11]:
input_quantizer = quantized_bits(14, 6, alpha=1)
output_quantizer = quantized_bits(18,14,alpha=1)

Y = data_file['data']["Background_data"]["Test"]["DATA"]
Y.shape[0] = 4511092

In [12]:
data = data_file['data']["Signal_data"]["GluGluHToGG_M-125"]["DATA"]

In [9]:
data.shape

(194433, 19, 3)

In [49]:
hls_model.graph.get('dot').__dir__()

['model',
 'name',
 'index',
 'inputs',
 'outputs',
 'attributes',
 'weights',
 'variables',
 'types',
 'code',
 'channels_last_converted',
 '_expected_attributes',
 '_wrapped',
 '__module__',
 '__doc__',
 'initialize',
 'expected_attributes',
 '__init__',
 'class_name',
 'set_attr',
 'get_attr',
 '_validate_attributes',
 '_wrap_precision_to_type',
 '_set_accum_t',
 'get_input_node',
 'get_input_variable',
 'get_output_use_map',
 'get_output_nodes',
 'get_output_variable',
 'get_weights',
 'get_variables',
 'add_output_variable',
 'add_weights',
 'add_bias',
 'add_weights_variable',
 'get_layer_precision',
 '__dict__',
 '__weakref__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__new__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [76]:
hls_model.graph.get('dot').get_input_node()

In [83]:
# HLS Tracing
trace = hls_model.trace(data[4].reshape(1,-1))

Recompiling myproject with tracing
Writing HLS project


Done


In [84]:
trace

(array([164.6875]),
 {'subtract': array([[ -0.125 ,   0.    ,  -1.25  ,   0.    ,  -2.6875,  -2.875 ,
            0.5   , -11.75  ,  -0.1875,  -3.0625]]),
  'dot': array([[164.6875]])})

In [26]:
vec = np.array([-0.125   ,   0.03125 ,  -1.203125,   0.046875,  -2.6875  ,
           -2.828125,   0.53125 , -11.734375,  -0.15625 ,  -3.015625])
print(vec)
print((vec*vec).sum())

[ -0.125      0.03125   -1.203125   0.046875  -2.6875    -2.828125
   0.53125  -11.734375  -0.15625   -3.015625]
163.783447265625


In [30]:
ind = '2'

In [31]:
sample_indices = np.random.choice(data.shape[0], 2048, False)
sample_indices.sort()
sample_indices
X = data[sample_indices]
X_flat = X.reshape(X.shape[0],-1)

### HLS

In [32]:
from tqdm import tqdm

pred_hls = []
for i in tqdm(range(0,X_flat.shape[0])):
    pred_hls_ = hls_model.predict(input_quantizer(X_flat[i]).numpy())
    pred_hls.append(output_quantizer(pred_hls_).numpy())

100%|██████████| 2048/2048 [00:04<00:00, 445.42it/s]


In [33]:
pred_hls = np.stack(pred_hls)
print(pred_hls)

[[-103.125]
 [  93.   ]
 [  39.375]
 ...
 [-116.   ]
 [-110.   ]
 [ -87.25 ]]


In [34]:
pred_hls = pred_hls.reshape(-1)

In [35]:
np.save("./pred_GGHToGG_M125/preds_hls"+ind,pred_hls)

### QONNX

In [36]:
import qonnx.core.onnx_exec as oxe
from qonnx.core.modelwrapper import ModelWrapper

qonnx_model = ModelWrapper("init_model.onnx")

In [37]:
from tqdm import tqdm

pred_qonnx = []

for i in tqdm(range(0, X_flat.shape[0])):
    idict = {qonnx_model.graph.input[0].name: input_quantizer(X[i]).numpy().reshape(1,57)}
    odict = oxe.execute_onnx(qonnx_model, idict, False)
    pred_ = output_quantizer(odict[qonnx_model.graph.output[0].name])
    pred_qonnx.append(pred_)

100%|██████████| 2048/2048 [27:11<00:00,  1.26it/s]


In [38]:
pred_qonnx_np = np.array(pred_qonnx)
pred_qonnx_np.shape
pred_qonnx_np = pred_qonnx_np.reshape(-1)

In [39]:
np.save("./pred_GGHToGG_M125/preds_qonnx_init"+ind,pred_qonnx_np)

In [40]:
del qonnx_model
qonnx_model = ModelWrapper("mod_model.onnx")

In [41]:
from tqdm import tqdm

pred_qonnx = []

for i in tqdm(range(0, X_flat.shape[0])):
    idict = {qonnx_model.graph.input[0].name: input_quantizer(X[i]).numpy().reshape(1,57)}
    odict = oxe.execute_onnx(qonnx_model, idict, False)
    pred_ = output_quantizer(odict[qonnx_model.graph.output[0].name])
    pred_qonnx.append(pred_)

100%|██████████| 2048/2048 [37:34<00:00,  1.10s/it]


In [42]:
pred_qonnx_np = np.array(pred_qonnx)
pred_qonnx_np.shape
pred_qonnx_np = pred_qonnx_np.reshape(-1)

In [43]:
np.save("./pred_GGHToGG_M125/preds_qonnx_mod"+ind,pred_qonnx_np)

### qKeras

In [44]:
qk_pred_ = model.predict(input_quantizer(X_flat))

64/64 [==============================] - 1s 5ms/step


In [45]:
qk_pred = output_quantizer(qk_pred_)
qk_pred = qk_pred.numpy()
qk_pred.shape

(2048, 1)

In [46]:
np.save("./pred_GGHToGG_M125/preds_qkeras"+ind, qk_pred.reshape(-1))